In [ ]:
# Asyncio loop for notebook execution in github action
import asyncio

try:
    asyncio.get_running_loop()
except RuntimeError:
    asyncio.set_event_loop(asyncio.new_event_loop())

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Snowflake initialization
from snowflake.snowpark import Session


# Définir ta configuration
connection_parameters = {
    "account": "ZICSLXO-IL01773",   # ex: "xy12345.eu-central-1"
    "user": "GITHUB",
    "password": "t9qGJYBJ6ctSgJugit",
    "warehouse": "COMPUTE_WH",
    "database": "RAW_DATA",
    "schema": "AMAZING_DATA"
}

# Créer la session
session = Session.builder.configs(connection_parameters).create()

In [ ]:
table = session.table("TRANSFORMED_DATA.AMAZING_DATA.CLIENT_EVENTS_CLEAN")

In [ ]:
table.show()

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

from sklearn.metrics import pairwise_distances
import numpy as np

# Récupérer un échantillon représentatif (ici le premier million de lignes)
sample_df = session.table("TRANSFORMED_DATA.AMAZING_DATA.CLIENT_EVENTS_CLEAN").limit(100000).to_pandas()

scaler = RobustScaler()
sample_scaled = scaler.fit_transform(sample_df)

pca = PCA(n_components=3)
pca.fit(sample_scaled)

sample_scaled = scaler.transform(sample_df)
sample_pca = pca.transform(sample_scaled)

In [ ]:
import joblib

df = session.table("TRANSFORMED_DATA.AMAZING_DATA.CLIENT_EVENTS_CLEAN").to_pandas()

df_scaled = scaler.transform(df)
df_pca = pca.transform(df_scaled)

In [ ]:
import joblib

local_path = "/tmp/model.joblib"
session.file.get("@clustering_model/model.joblib", local_path)

model = joblib.load("/tmp/model.joblib/model.joblib")

In [ ]:
pred_labels = model.predict(df_pca)

labels_pred = model.predict(df_pca)
df["cluster"] = labels_pred

In [ ]:
df.info()

In [ ]:
df_clusters = session.create_dataframe(df)

# Sauvegarde dans une nouvelle table
df_clusters.write.save_as_table(
    "TRANSFORMED_DATA.AMAZING_DATA.PREDICTED_DATA",
    mode="overwrite"
)